In [34]:
import pennylane as qml
from pennylane import numpy as np
import math
from pennylane.optimize import AdamOptimizer

In [35]:
num_qubit_factoring = 11+1
dev1 = qml.device('default.qubit', wires=num_qubit_factoring)
@qml.qnode(dev1)
def oracle_circuit():
    qml.Hadamard(wires=1)  
    qml.Hadamard(wires=4)  
    
    #C1
    qml.CNOT(wires=[1,4])
    qml.CNOT(wires=[4,7])
    qml.CNOT(wires=[1,4])
    
    #C2
    qml.Toffoli(wires=[1,5,8])
    qml.Toffoli(wires=[2,4,9])
    qml.PauliX(wires=[8])
    qml.PauliX(wires=[9])
    qml.Toffoli(wires=[8,9,10])
    qml.PauliX(wires=[8])
    qml.PauliX(wires=[9])
    qml.Toffoli(wires=[1,5,8])
    qml.Toffoli(wires=[2,4,9])
    
    #AND1
    qml.Toffoli(wires=[7,10,11])

    qml.CNOT(wires=[11,0])
    return qml.probs([i for i in range(0,6)])
    

oracle_result = oracle_circuit()
norm = np.linalg.norm(oracle_result)
oracle_result /= norm
oracle_result = oracle_result.tolist()
print(oracle_result)

print('Oracle:')
print(qml.draw(oracle_circuit)())

[0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Oracle:
 0: ─────────────────────────────────────╭X─┤ ╭Probs
 1: ──H─╭●────╭●─╭●─────────────╭●───────│──┤ ├Probs
 2: ────│─────│──│──╭●──────────│──╭●────│──┤ ├Probs
 3: ────│─────│──│──│───────────│──│─────│──┤ ├Probs
 4: ──H─╰X─╭●─╰X─│──├●──────────│──├●────│──┤ ├Probs
 5: ───────│─────├●─│───────────├●─│─────│──┤ ╰Probs
 7: ───────╰X────│──│───────────│──│──╭●─│──┤       
 8: ─────────────╰X─│───X─╭●──X─╰X─│──│──│──┤       
 9: ────────────────╰X──X─├●──X────╰X─│──│──┤       
10: ──────────────────────╰X──────────├●─│──┤       
11: ──────────────────────────────────╰X─╰●─┤       


In [36]:
def vqs_factoring(num_qubits_vqs,n_layers,oracle,n_shots=None,device_name1='default.qubit' ,device_name2='default.qubit' ):
    print('run VQS n_layers=',n_layers,'device1 = ',device_name1,'device2 = ',device_name2)
    val_global = []
    # vqs
    num_qubit_vqs = 1+num_qubits_vqs
    eps_val_q = 1/math.sqrt(2**num_qubit_vqs)/100
    eps_val = min(1e-10, eps_val_q)
    tiny_change_threshold = 1e-4
    cnt_threshold_no_change = 5
    

    N = 2**(num_qubit_vqs-2)
    normal_val = math.sqrt(1/N)


    def oracle_builder_for_HT_HTZ():
        initial_state_0_phi1  = oracle_result + [0]*len(oracle_result) 
        qml.QubitStateVector(np.array(initial_state_0_phi1), wires=range(num_qubits_vqs))

    def oracle_builder_for_no_HT_HTZ():
        qml.QubitStateVector(np.array(oracle), wires=range(num_qubits_vqs-1))


    def layer_t3_no_HT(theta, qubit_posi):
        # type-2 layer
        # length of theta: (num_qubit_vqs-1)*2
        # length of qubit_posi: num_qubit_vqs-1
        # number of wires: num_qubit_vqs
        for i in range(num_qubit_vqs-1):
            qml.RY(theta[i], wires=(qubit_posi[i]))
        for i in np.arange(0, num_qubit_vqs-2, 2):
            qml.CNOT(wires=(qubit_posi[i], qubit_posi[i+1]))  # CNOT struct3
        for i in range(num_qubit_vqs-1):
            qml.RY(theta[i+num_qubit_vqs-1], wires=(qubit_posi[i]))
        for i in np.arange(1, num_qubit_vqs-2, 2):
            qml.CNOT(wires=(qubit_posi[i], qubit_posi[i+1]))  # CNOT struct3
        qml.CNOT(wires=(qubit_posi[-1], qubit_posi[0]))  # CNOT struct3


    def layer_t3_with_HT(theta, num_qubit_vqs):
        # type-2 layer
        # length of theta: (num_qubit_vqs-1)*2
        # number of wires: num_qubit_vqs
        for i in range(num_qubit_vqs-1):
            qml.CRY(theta[i], wires=(0, i+1))
        for i in np.arange(0, num_qubit_vqs-2, 2):
            qml.Toffoli(wires=(0, i+1, i+2))  # CCNOT struct3

        for i in range(num_qubit_vqs-1):
            qml.CRY(theta[i+num_qubit_vqs-1], wires=(0, i+1))
        for i in np.arange(1, num_qubit_vqs-2, 2):
            qml.Toffoli(wires=(0, i+1, i+2))  # CCNOT struct3
        qml.Toffoli(wires=(0, num_qubit_vqs-1, 1))  # CCNOT struct3


    dev_with_HT = qml.device(device_name1, wires=num_qubits_vqs+1,shots=n_shots)
    @qml.qnode(dev_with_HT)
    def quantum_circuit_with_HT(theta):
        oracle_builder_for_HT_HTZ()
        qml.Hadamard(0)
        for theta_i in theta:
            layer_t3_with_HT(theta_i, num_qubit_vqs)
        qml.Hadamard(0)
        
        return qml.expval(qml.PauliZ(0))

    dev_with_HTZ = qml.device(device_name1, wires=num_qubits_vqs+1,shots=n_shots)
    @qml.qnode(dev_with_HTZ)
    def quantum_circuit_with_HTZ(theta):
        oracle_builder_for_HT_HTZ()
        qml.Hadamard(0)
        for theta_i in theta:
            layer_t3_with_HT(theta_i, num_qubit_vqs)
        qml.CZ([0, 1])
        qml.Hadamard(0)
        return qml.expval(qml.PauliZ(0))


    dev_no_HT_S = qml.device(device_name2, wires=num_qubits_vqs,shots=n_shots)
    @qml.qnode(dev_no_HT_S)
    def quantum_circuit_no_HT_return_state(theta):
        oracle_builder_for_no_HT_HTZ()
        for theta_i in theta:
            layer_t3_no_HT(theta_i, list(range(num_qubit_vqs-1)))
        return qml.state()


    def objective_fn(theta):
        val1_1 = quantum_circuit_with_HT(theta)
        val1_2 = quantum_circuit_with_HTZ(theta)
        val1_1 = val1_1/normal_val
        val1_2 = val1_2/normal_val
        obj = -0.5*(val1_1 - val1_2)
        val_global.append(
            [val1_1._value.tolist(), val1_2._value.tolist(), obj._value.tolist()])
        
        return obj

    iter_max = 300  # 300
    num_of_layers = n_layers
    obj_list_rep = []
    theta_list = []
    iter_terminate_list = []
    print_flag = True
    optimizer = AdamOptimizer(0.05, beta1=0.9, beta2=0.999)
    theta = np.random.uniform(
        0, 2*math.pi, size=(num_of_layers, 2*(num_qubit_vqs-1)), requires_grad=True)

    obj_list = []
    tiny_change_cnt = 0
    break_flag = False
    iter_terminate = iter_max
    for iter in range(1, iter_max+1):
        theta, obj = optimizer.step_and_cost(objective_fn, theta)
        val1_1 = val_global[-1][0]
        val1_2 = val_global[-1][1]
        if iter >= 2:
            val1_1_old = val_global[-2][0]
            val1_2_old = val_global[-2][1]
        else:
            val1_1_old = 999
            val1_2_old = 999
        val1 = val1_1 - val1_2
        val1_old = val1_1_old - val1_2_old
        if abs(val1) > eps_val:  # eps_val=1e-10
            if abs((val1-val1_old)/val1) < tiny_change_threshold:  # 1e-3
                tiny_change_cnt += 1
            else:
                tiny_change_cnt = 0
        if tiny_change_cnt >= cnt_threshold_no_change:  # no change for a consequtive of 5 iterations, then break
            break_flag = True
        if (iter == 1 or iter % 10 == 0 or iter == iter_max) and print_flag:
            #TODO ? 
            if (np.isclose(val1_1-val1_2,0)):
                return False
            print(f'iter={iter:3d} :: obj={obj:12.8f} :: val1_1={val1_1:12.8f} :: val1_2={val1_2:12.8f} :: -0.5*(val1_1 - val1_2)={-0.5*(val1_1 - val1_2):12.8f}')

        obj_list.append(obj)
        if break_flag:
            iter_terminate = iter
            break
    theta_list.append(theta)
    obj_list_rep.append(obj_list)

    val_global = []  # reset to empty

    # display the amplified state
    state = quantum_circuit_no_HT_return_state(theta)
    iter_terminate_list.append(iter_terminate)
    return state

In [37]:
num_qubits_vqs = 6+1
n_layers = 3
results = vqs_factoring(num_qubits_vqs=num_qubits_vqs,n_layers=n_layers,oracle=oracle_result)

run VQS n_layers= 3 device1 =  default.qubit device2 =  default.qubit
iter=  1 :: obj= -0.03918170 :: val1_1= -0.02010199 :: val1_2= -0.09846539 :: -0.5*(val1_1 - val1_2)= -0.03918170
iter= 10 :: obj= -3.49096972 :: val1_1=  4.63812269 :: val1_2= -2.34381676 :: -0.5*(val1_1 - val1_2)= -3.49096972
iter= 20 :: obj= -5.03937261 :: val1_1=  5.19062506 :: val1_2= -4.88812016 :: -0.5*(val1_1 - val1_2)= -5.03937261
iter= 30 :: obj= -5.36901709 :: val1_1=  5.17290099 :: val1_2= -5.56513320 :: -0.5*(val1_1 - val1_2)= -5.36901709
iter= 40 :: obj= -5.52420820 :: val1_1=  5.35786341 :: val1_2= -5.69055299 :: -0.5*(val1_1 - val1_2)= -5.52420820
iter= 50 :: obj= -5.61251914 :: val1_1=  5.46872687 :: val1_2= -5.75631142 :: -0.5*(val1_1 - val1_2)= -5.61251914
iter= 60 :: obj= -5.63436755 :: val1_1=  5.50625349 :: val1_2= -5.76248161 :: -0.5*(val1_1 - val1_2)= -5.63436755
iter= 70 :: obj= -5.64406744 :: val1_1=  5.50683503 :: val1_2= -5.78129985 :: -0.5*(val1_1 - val1_2)= -5.64406744
iter= 80 :: obj= -

In [39]:
results_probs = [(np.linalg.norm(res))**2 for res in results]
finded_solutions={}
for i in range(len(results_probs)):
    if(results_probs[i]>0.1):
        binary_i = '{0:06b}'.format(i)
        finded_solutions[binary_i[1:5]]=results_probs[i]
print(finded_solutions)
counter = 0
for key in finded_solutions.keys():
    if counter!=0:
        print('OR')
    print(fr'p1={key[0]} & p2={key[1]} & q1={key[2]} & q2={key[3]}')
    counter+=1

{'0001': 0.495203374159862, '1000': 0.5026178541500761}
p1=0 & p2=0 & q1=0 & q2=1
OR
p1=1 & p2=0 & q1=0 & q2=0
